In [1]:
%matplotlib inline
#imports needed for plotting and animation
import matplotlib.pyplot as plt
import autograd.numpy as np
import hpc 
import pykat as pk
import skimage
import scipy
from scipy import fft

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
from matplotlib import animation
from IPython.display import HTML


from autograd import elementwise_grad, value_and_grad
from scipy.optimize import minimize
from collections import defaultdict
from itertools import zip_longest
from functools import partial

from IPython.display import HTML

                                              ..-
    PyKat 1.1.310         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



This code was written as a method to optimize a FLIR BlackFlyS IR camera which the LIGO Lab at the University of Florida is using with two 1064nm lasers to test and develop the concept of a heterodyne phase camera. 

The optimization method developed for this purpose can loosely be defined as a type of Markov Chain Monte Carlo (MCMC) method. The 'Monte Carlo' aspect involves seeding the parameter space with many random starting points. The 'Markov Chain" aspect involves an 'intelligent' random walk, wherein for each random step the chain can move to, the chain's decision engine must decide if moving to the new position will decrease the figure of merit (FOM) in the case of minimization. If the new position does decrease the FOM, the chain continues on its quest for lower values of the FOM. If it helps to visualize, I like to think of the chains as snakes; hence I have named functions with the snake theme. Another way to visualize what is occuring is to watch the short movie hosted in the same repository with this script (https://github.com/pfulda/pjflab/blob/master/Simulation_Output/MCMC_optimization_toy_model_2D_video.mp4) which shows the MCMC minimization for an analytical function and small parameter space. 



For a user of this tool, I am providing some tips and general information about the method and the results you can expect:

1) In a rather unorthodox manner, the FOM was chosen to be the minimum value of $-<SNR>^2$ (negated average 'amplitude-SNR' squared). Why?:  
  
a) Negated because a previous model used to demonstrate the effectiveness of the method was a minimizaiton

b) The averaging is the average value of SNR in a single pixel over a 10x10 cluster of pixels through a times series of 400 images (i.e. sum SNR in all 100 pixels, then divide by 100)

c) We square the 'amplitude-SNR' so that we have 'power-SNR'


2) This method assumes the parameter space is 3D; with the three dimensions being camera exposure time, laser power, and camera gain setting.

3) The laser power, which is specified by 'power1' in the function BeamChoice is the power in watts provided to each of the simulated laser beams, not the total power incident on the camera.

4) Exposure time has units of seconds.

5) Gain has units of dB.

6) There are certain parameters which the user should be aware of and change if desired. These include:

a) the numpy arrays which determine the length and values on each of the parameter space's dimensions ('exp_bounds', 'pwr_bounds', 'gain_bounds')

b) the python list objects which contain the allowed step sizes ('StepSet_exp', 'StepSet_pwr', StepSet_gain')

c) 'NumPaths' is an input to 'CreateAllPaths' and determines the number of snakes you want to search your parameter space and 'Numberiterations' is an input to 'CreateAllPaths' which determines the number of steps each snake is 'allowed' to take (this is a tradeoff, snakes which start in 'poor' locations in the parameter space should be terminated quickly to reduce unnecesary computational expense (small value of 'Numberiterations' desired), but you would want snakes seeded in a 'good' location to have enough step opportunities to reach the global minimum (large value of 'Numberiterations' desired)).

d) The variable 'delta' in 'SnakeChoice' allows for new step FOM values to be close to, but not exactly less than the current position. This is useful for our computation of the FOM because each computation does not yield the exact same value (i.e. there is always some non-zero standard deviation).

e) In the function 'CreateFullPath_i' there is a line of code, 'if (-0.5 < (Zvec[index_i]-Zvec[index_i+1]) < 0.5):'. The two values, -0.5 and 0.5, can be changed to any values you desire. These are specified to make sure the snake is making sufficient progress in its minimization trajectory. If the difference between two FOM step calculations (i.e. Zvec values) is not between these bounds, then the path will terminate.

7) The primary function for calculating the FOM subject to different input parameters is 'ParamChoices_SNR'.

8) The primary function for the MCMC method 'decision-making' is 'SnakeChoice'.

9) The primary function to run the entire MCMC optimization is 'CreateAllPaths'.

10) The primary function for quick data analysis of the optimization is 'SNRAnalysis'.

11) The chunk of code beneath 'SNRAnalysis' is a diagnostic to show the standard deviation of computing the FOM many times with the same input parameters. So for instance, after running the method with 'CreateAllPaths', and retrieving the parameter inputs from 'SNRAnalysis', you could put those parameter inputs into the stddev calculation code to have a value of the standard deviation you can expect for the minimum FOM calculated.

12) The HPC.py package was built by Mauricio Diaz-Ortiz at UF, with later development by other members of the UF LIGO group. For questions on the package either email mdiazort@ufl.edu or see the Github account which hosts HPC.py, https://github.com/Mo-Rice/hpc_sim.

If you have questions on this method, you can contact me at https://kadenjloring.wordpress.com/.

I hope this is of some use to you!

Chunks below set up variables and functions necessary for camera optimization:

In [2]:
#function to define a given sensor (say, sensor_i) according to it's allowed exposure time
#exposure_time=4e-6 was default in Mauricio's example
#ideally, gain will be altered through this function also
#for whole CMOS x_resolution = 728, y_resolution = 544
#x_resolution (and y) will grow from center, as defined in HPC.py
def SensorChoice(exposure_time, gain_val):
    #define sensor
    sensor = hpc.CMOS_sensor(pixel_pitch=6.9e-6, x_resolution=10, y_resolution=10, exposure_time=exposure_time, quantum_eff=0.03, gain = gain_val, pixel_well_depth=22187)
    
    #define x and y arrays
    x_array = sensor.x_array
    y_array = sensor.y_array
    
    return sensor, x_array, y_array

In [3]:
#function to define a single beam power to be used for both beams,
#Mauricio used .1e-3 as the power for both beams in his example
def BeamChoice(power1, x_array, y_array):
    beam1 = hpc.beam(power=power1, w0=1e-3, z=0, spatial="gauss")
    beam1_amp = beam1.generate_amplitude_map(x_array, y_array)
    #slight curvature given to beam2, but same size
    beam2_curved = hpc.beam(power=power1, w0=1e-3, z=.3, spatial="gauss")
    beam2_curved_amp = beam2_curved.generate_amplitude_map(x_array, y_array)
    
    return beam1_amp, beam2_curved_amp

In [4]:
#function defines length of time for capturing images, interval between captures
#adds shot noise to signal and return the data at the chosen pixel value (say, [290][350]) as a time series
def GenerateAllPixels(sensor, beam1_amp, beam2_curved_amp):
    #if no_images is adjusted, collection time must be adjusted also
    t = hpc.time_array(no_images=400, collection_time=10)
    beat_note = hpc.generate_beatnote(beam1_amp, beam2_curved_amp, 10, t)
    #make sure if you change no_images, you also change num_images 
    n_beatnote, SatBool = sensor.capture(np.around(beat_note,2), bitdepth=12)
    
    return t, n_beatnote, SatBool

In [5]:
def PullPixelData(rowi, coli, n_beatnote):
    #pulls out value at pixel [rowi][coli] of image i
    pixel_data = [i[rowi][coli] for i in n_beatnote]
    return pixel_data

In [6]:
def SNR(t,pixel_data):
    N = len(t)
    fs = 40
    ft = 2*scipy.fft(pixel_data)/N
    h = ft[0:N//2]
    signal = np.abs(h[100])
    noise = np.sum(np.abs(h[1:99])) + np.sum(np.abs(h[101:200]))
    SNR = signal/noise
    
    return SNR

In [20]:
#this will take the place of f in the toy model 
def AvgSNR2(sensor, x_array, y_array, beam1_amp, beam2_curved_amp):
    """
    neg_Avg_SNR2: return negative value of the squared avg SNR amongst the calculated pixels
    x_array: numpy array generated from defining sensor
    y_array: numpy array generated from defining sensor
    beam1_amp: numpy array generated by specifying beam power
    beam2_curved_amp: numpy array generated by specifying curvature and beam power
    """
    SNRs = []
    t, n_beatnote, SatBool = GenerateAllPixels(sensor, beam1_amp, beam2_curved_amp)
    for i in range(len(x_array)):
        for j in range(len(y_array)):
            pixel_data = PullPixelData(rowi=i, coli=j, n_beatnote=n_beatnote)
            SNRi = SNR(t=t, pixel_data=pixel_data)
            SNRs.append(SNRi)
    neg_Avg_SNR2 = -1*(sum(SNRs)/len(SNRs))**2
    #so as to ensure saturated (-<SNR>^2) do not dominate the results....just set equal to 0
    if SatBool == 1:
        neg_Avg_SNR2 = 0
    
    return neg_Avg_SNR2

In [21]:
  def ParamChoices_SNR(exposure_time, power, gain_val):
    """
    exposure: value provided for exposure time
    power: value which specifies power for beam1 (same power assigned to beam2)
    SNR_array: return value containing negative squared average SNR values (square to make it in units of power, 
    negative to conform with current MCMC optimization method)
    """
    sensor, x_array, y_array = SensorChoice(exposure_time=exposure_time, gain_val=gain_val)
    #print("exposure_time: " + str(exposure_time))
    #define beam amps
    beam1_amp, beam2_curved_amp = BeamChoice(power1=power, x_array=x_array, y_array=y_array)
    #print("power1: " + str(power))
    #compute averaged SNR, then square and make negative
    SNR_Avg_2 = AvgSNR2(sensor=sensor,x_array=x_array, y_array=y_array, beam1_amp=beam1_amp, beam2_curved_amp=beam2_curved_amp)

    return SNR_Avg_2

Chunks below set up basic global parameters:

In [22]:
#Camera (don't think this necessary if we aren't plotting whole parameter space)
#note, exposure time definitely can't exceed 0.025s based on 40fps requirement
#exposure time takes place of x in 2D optimization, power takes place of y
#gain will take place of 3rd parameter (i.e. third entry in Boundaries)
#gain is in [dB]
#from experience in lab, even with a few filters, 10mW of total power would probably saturate the sensor
#first entry in each parameter boundary is lower bound, next is upper bound
#Boundaries = [[4e-6, 1e-4, 1e-5], [1e-6, 1e-3, 1e-5], [0, 15, 1]]
exp_bounds = np.arange(4e-6, 1e-5, 1e-7)
pwr_bounds = np.arange(1e-7, 1e-4, 1e-6)
gain_bounds = np.arange(0,5,0.05)

In [23]:
#Camera parameter space dimensions 
ParamSpcDims=3

In [24]:
#camera 
#these are the choices allowed for new MCMC steps
StepSet_exp = [-0.5e-6, 0, 0.5e-6]
StepSet_pwr = [-1e-7, 0, 1e-7]
StepSet_gain = [-0.2,0,0.2]

Chunks below define functions which build up data necessary for animation and processing (deleted Terrain(Boundaries)):

In [25]:
def CreatePathNames(NumPaths):
    #set up path names
    path_iter = []
    path_base = "path"
    for i in range(0,NumPaths):
        path_name = path_base + "_" + str(i)
        path_iter.append(path_name)
    return path_iter


In [26]:
def AppendPathStart(path_iter, x0s):
    #add random (x,y) starting points to individual paths which are initially held as lists in a dict
    paths_ = defaultdict(list)
    iii=0
    for path in path_iter:
        paths_[path].append(x0s[:,iii])
        iii+=1
    return paths_

In [27]:
#create random initial coords for each of your N number of paths
def SnakePath_Start(NumPaths, ParamSpcDims, exp_bounds, pwr_bounds, gain_bounds):
    #create random (x,y) starting points:
    #initialize empty list to place random (x,y) starting points
    x0s=[]
    #run for loop to append random (x,y,q) to list as numpy arrays, these coords are called x0i
    for i in range(NumPaths):
        x0i = np.array([np.random.choice(exp_bounds), np.random.choice(pwr_bounds), np.random.choice(gain_bounds)])
        #print("this is x0" + str(i) + ": " + str(x0i))
        x0s.append(x0i)
    #print("this is x0s:" +str(x0s))
    #print("shape:"+str(np.shape(x0s)))
    x0s = np.array(x0s).T
    print("this is x0s (post T):" +str(x0s))
    print("shape:"+str(np.shape(x0s)))
    path_iter = CreatePathNames(NumPaths)
    paths_ = AppendPathStart(path_iter, x0s)
    
    return x0s, paths_, path_iter


In [29]:
#This is the Markov Chain portion of the MCMC, you are effectively only moving to a new coords
#iff it will decrease the z value
#try three small increment comparisons, if neither, take kamikazi leap for new position in parameter space
#StepSet should be list of possible steps, one set for each of the camera parameters 
#X0i should be 3X1 coords
#ParamSpcDims should be 3
#AvgSNR2 should be function to be minimized
def SnakeChoice(paths_, StepSet_exp, StepSet_pwr, StepSet_gain, ParamSpcDims, ParamChoices_SNR, path_i, index_i):
    CoordsPostChoice = np.zeros((1,ParamSpcDims))
    
    x0 = paths_['path_'+str(path_i)][index_i][0]
    #print("this is x0:" +str(x0) + "\n")
    y0 = paths_['path_'+str(path_i)][index_i][1]
    #print("this is y0:" +str(y0) + "\n")
    q0 = paths_['path_'+str(path_i)][index_i][2]
    print("Initial (x,y)= (" + str(x0) + "," + str(y0) + "," +str(q0) + ")")
    #initial Z value at beginning of step choice sequence
    Zvalue0 = ParamChoices_SNR(x0,y0,q0)
    print("Initial Z value:" + str(Zvalue0))
    #random addition to x0, y0
    Plusx0 = np.random.choice(a=StepSet_exp)
    Plusy0 = np.random.choice(a=StepSet_pwr)
    Plusq0 = np.random.choice(a=StepSet_gain)
    #ensures no (0,0) additions to (x,y)
    if Plusx0==0 and Plusy0==0 and Plusq0==0:
        Plusy0 = np.random.choice(a=[StepSet_pwr[0], StepSet_pwr[2]])
    print("Plusx0: " + str(Plusx0))
    print("Plusy0: " + str(Plusy0))
    print("plusq0: " + str(Plusq0))
    #potential new coords
    xn0 = x0+Plusx0
    yn0 = y0+Plusy0
    qn0 = q0+Plusq0
    #check on boundary conditions (i.e. keep snakes on bounded space)
    if xn0 > exp_bounds[-1]:
        xn0 = xn0 - 2*Plusx0
    if xn0 < exp_bounds[0]:
        xn0 = xn0 - 2*Plusx0
    if yn0 > pwr_bounds[-1]:
        yn0 = yn0 - 2*Plusy0
    if yn0 < pwr_bounds[0]:
        yn0 = yn0 - 2*Plusy0
    if qn0 > gain_bounds[-1]:
        qn0 = qn0 - 2*Plusq0
    if qn0 < gain_bounds[0]:
        qn0 = qn0 - 2*Plusq0
    #comparison Z value
    Zvalue_n0 = ParamChoices_SNR(xn0,yn0,qn0)
    print("Z value from step attempt 1: " + str(Zvalue_n0) + ", with (x,y,q): (" + str(xn0) + "," + str(yn0) + "," + str(qn0) +")")
    #round-off delta, will artificially increase Zvalue0 so that comparison has a better chance
    delta = 1e-2
    #test: Y: try another random step; N: new coords accepted bc new Z is less than original Z+delta
    if Zvalue0 + delta < Zvalue_n0:
        Plusx1 = np.random.choice(a=StepSet_exp)
        Plusy1 = np.random.choice(a=StepSet_pwr)
        Plusq1 = np.random.choice(a=StepSet_gain)
        #ensures no (0,0) additions to (x,y)
        if Plusx1==0 and Plusy1==0 and Plusq1==0:
            Plusy1 = np.random.choice(a=[StepSet_pwr[0], StepSet_pwr[2]])
        print("Plusx1: " + str(Plusx1))
        print("Plusy1: " + str(Plusy1))
        print("Plusy1: " + str(Plusq1))
        xn1 = x0 + Plusx1
        yn1 = y0 + Plusy1
        qn1 = q0 + Plusq1
        #ensures coords stay within bounds
        if xn1 > exp_bounds[-1]:
            xn1 = xn1 - 2*Plusx1
        if xn1 < exp_bounds[0]:
            xn1 = xn1 - 2*Plusx1
        if yn1 > pwr_bounds[-1]:
            yn1 = yn1 - 2*Plusy1
        if yn1 < pwr_bounds[0]:
            yn1 = yn1 - 2*Plusy1
        if qn1 > gain_bounds[-1]:
            qn1 = qn1 - 2*Plusq1
        if qn1 < gain_bounds[0]:
            qn1 = qn1 - 2*Plusq1
    
        Zvalue_n1 = ParamChoices_SNR(xn1,yn1,qn1)
        print("Z value from step attempt 2: " + str(Zvalue_n1) + ", with (x,y,q): (" + str(xn1) + "," + str(yn1) + "," + str(qn1) + ")")
        #test: Y: try another random step; N: return xn1, yn1
        if Zvalue0 + delta < Zvalue_n1:
            Plusx2 = np.random.choice(a=StepSet_exp)
            Plusy2 = np.random.choice(a=StepSet_pwr)
            Plusq2 = np.random.choice(a=StepSet_gain)
            #ensures no (0,0) additions to (x,y)
            if Plusx2==0 and Plusy2==0:
                Plusy2 = np.random.choice(a=[StepSet_pwr[0], StepSet_pwr[2]])
            print(Plusx2)
            print(Plusy2)
            xn2 = x0 + Plusx2
            yn2 = y0 + Plusy2
            qn2 = q0 + Plusq2
            #arbitrary choice to send x back by 2*random x step if exceeds bounds
            if xn2 > exp_bounds[-1]:
                xn2 = xn2 - 2*Plusx2
            if xn2 < exp_bounds[0]:
                xn2 = xn2 - 2*Plusx2
            if yn2 > pwr_bounds[-1]:
                yn2 = yn2 - 2*Plusy2
            if yn2 < pwr_bounds[0]:
                yn2 = yn2 - 2*Plusy2
            if qn2 > gain_bounds[-1]:
                qn2 = qn2 - 2*Plusq2
            if qn2 < gain_bounds[0]:
                qn2 = qn2 - 2*Plusq2
          
            Zvalue_n2 = ParamChoices_SNR(xn2,yn2,qn2)
            print("Z value from step attempt 3: " + str(Zvalue_n2) + ", with (x,y,q ): (" + str(xn2) + "," + str(yn2) + "," +str(qn2) + ")")
            #test: Y: no big jump attempt, N: return xn2, yn2
            if Zvalue0 + delta < Zvalue_n2:
                PlusBIGx = 10*np.random.choice(a=StepSet_exp)
                PlusBIGy = 10*np.random.choice(a=StepSet_pwr)
                PlusBIGq = 2*np.random.choice(a=StepSet_gain)
                if PlusBIGx==0 and PlusBIGy==0:
                    PlusBIGy = 5*np.random.choice(a=[StepSet_pwr[0], StepSet_pwr[2]])
                print(PlusBIGx)
                print(PlusBIGy)
                xnBIG = x0 + PlusBIGx
                ynBIG = y0 + PlusBIGy
                qnBIG = q0 + PlusBIGq
                #arbitrary choice to send x back by 2*random x step
                if xnBIG > exp_bounds[-1]:
                    xnBIG = xnBIG - 2*PlusBIGx
                if xnBIG < exp_bounds[0]:
                    xnBIG = xnBIG - 2*PlusBIGx
                if ynBIG > pwr_bounds[-1]:
                    ynBIG = ynBIG - 2*PlusBIGy
                if ynBIG < pwr_bounds[0]:
                    ynBIG = ynBIG - 2*PlusBIGy
                if qnBIG > gain_bounds[-1]:
                    qnBIG = qnBIG - 2*PlusBIGq
                if qn0 < gain_bounds[0]:
                    qnBIG = qnBIG - 2*PlusBIGq
                Zvalue_nBIG = ParamChoices_SNR(xnBIG, ynBIG, qnBIG)
                print("Z value from BIG JUMP final attempt: " + str(Zvalue_nBIG) + ", with (x,y,q): (" + str(xnBIG) + "," + str(ynBIG) + "," + str(qnBIG) + ")")
                if Zvalue0 + delta < Zvalue_nBIG:
                    CoordsPostChoice[:,0]=x0
                    CoordsPostChoice[:,1]=y0
                    CoordsPostChoice[:,2]=q0
                    #print("shape of coords: "+ str(np.shape(CoordsPostChoice)))
                    Zexit = np.asarray(Zvalue0).reshape((1,1))
                    paths_['path_'+str(path_i)]=np.vstack([paths_['path_'+str(path_i)],CoordsPostChoice])
                else:
                    CoordsPostChoice[:,0]=xnBIG
                    CoordsPostChoice[:,1]=ynBIG
                    CoordsPostChoice[:,2]=qnBIG
                    #print("shape of coords: "+str(np.shape(CoordsPostChoice)))
                    Zexit = np.asarray(Zvalue_nBIG).reshape((1,1))
                    paths_['path_'+str(path_i)]=np.vstack([paths_['path_'+str(path_i)],CoordsPostChoice])
            else:
                CoordsPostChoice[:,0]=xn2
                CoordsPostChoice[:,1]=yn2
                CoordsPostChoice[:,2]=qn2
                #print("shape of coords: "+str(np.shape(CoordsPostChoice)))
                Zexit = np.asarray(Zvalue_n2).reshape((1,1))
                paths_['path_'+str(path_i)]=np.vstack([paths_['path_'+str(path_i)],CoordsPostChoice])
        else: 
            CoordsPostChoice[:,0]=xn1
            CoordsPostChoice[:,1]=yn1
            CoordsPostChoice[:,2]=qn1
            #print("shape of coords: "+str(np.shape(CoordsPostChoice)))
            Zexit = np.asarray(Zvalue_n1).reshape((1,1))
            paths_['path_'+str(path_i)]=np.vstack([paths_['path_'+str(path_i)],CoordsPostChoice])
                         
    else:
        CoordsPostChoice[:,0]=xn0
        CoordsPostChoice[:,1]=yn0
        CoordsPostChoice[:,2]=qn0
        #print("shape of coords: "+str(np.shape(CoordsPostChoice)))
        Zexit = np.asarray(Zvalue_n0).reshape((1,1))
        paths_['path_'+str(path_i)]=np.vstack([paths_['path_'+str(path_i)],CoordsPostChoice])
    #decided I wanted something more like 2X1 than 1X2 for CoordsPostChoice
    return CoordsPostChoice, Zexit, paths_['path_'+str(path_i)];
   

In [30]:
#current_coords should be one of the 2X1 entries of 'paths_'; for ex. paths_['path_i']
def CreateFullPath_i(Numberiterations, paths_, path_i, StepSet_exp, StepSet_pwr, StepSet_gain):
    index_i=0
    BreakFlag=0
    Zvec = np.asarray(ParamChoices_SNR(paths_['path_'+str(path_i)][index_i][0],paths_['path_'+str(path_i)][index_i][1], paths_['path_'+str(path_i)][index_i][2] )).reshape((1,1))
    current_coords = paths_['path_'+str(path_i)]
    while (BreakFlag==0) and (index_i<Numberiterations):
        compare_coords = current_coords
        current_coords, Zexit, paths_['path_'+str(path_i)] = SnakeChoice(paths_,StepSet_exp, StepSet_pwr, StepSet_gain, ParamSpcDims,ParamChoices_SNR,path_i, index_i)
        Zvec=np.vstack([Zvec,Zexit])
        print("this is Zvec: " + str(Zvec))
        if (-0.5 < (Zvec[index_i]-Zvec[index_i+1]) < 0.5):
            BreakFlag = 1
            print("Break condition met, negligible difference between consecutive Z values\n")
            print("Current Coords: "+ str(current_coords)+", Zexit: " + str(Zexit) + ", iterations performed: " +str(index_i) + "\n")
            if np.array_equal(compare_coords,current_coords) == 1:
                print("\n Break condition tripped by unchanged coords!")
        if (index_i+1 == Numberiterations):
            BreakFlag = 1
            print("\n Break condition met by reaching iteration limit!")
        print("Difference in consecutive Z vals: " + str(Zvec[index_i]-Zvec[index_i+1]))
        index_i += 1
     
    return paths_['path_'+str(path_i)], Zvec

In [31]:
#function to create all i paths
def CreateAllPaths(NumPaths, StepSet_exp, StepSet_pwr, StepSet_gain, ParamSpcDims, Numberiterations):
    #set up initial random path coordinates
    x0s, paths_, path_iter = SnakePath_Start(NumPaths, ParamSpcDims=ParamSpcDims, exp_bounds=exp_bounds, pwr_bounds=pwr_bounds, gain_bounds=gain_bounds)
    ZvecTotal = []
    for path_i in range(0,NumPaths):
        paths_['path_'+str(path_i)], Zvec = CreateFullPath_i(Numberiterations=Numberiterations, paths_=paths_, path_i=path_i, StepSet_exp=StepSet_exp, StepSet_pwr=StepSet_pwr, StepSet_gain=StepSet_gain)
        ZvecTotal.append(Zvec)
    return paths_, ZvecTotal, path_iter, Numberiterations

In [41]:
def SNR_Analysis(ZvecTotal, paths_, exp_bounds, pwr_bounds, gain_bounds):
   #used to store the minimum -SNR^2 value from each seed
    mins=[] 
    #used to store the indices of the minimum -SNR^2 from each path
    mins_params = []
    #count number of saturated paths
    NumSatPaths = 0
    #which paths saturated
    SatPaths = []
    #find min in a path, append value to mins, append indx where the min occurred from the path
    for i in range(0,len(ZvecTotal)):
        min_i = np.min(ZvecTotal[i])
        minidx = np.where(ZvecTotal[i]==min_i)
        minidx = minidx[0]
        mins.append(min_i)
        mins_params.append(paths_['path_'+str(i)][-1])
        if ZvecTotal[i][-1] ==0:
            NumSatPaths += 1
            SatPaths.append(i)
            
            
        
    #SNR-type val
    AbsMin = min(mins)
    #integer index from mins, this is essentially telling us which ZvecTotal[i]
    WhichZvec = mins.index(AbsMin)
    #pull out the parameters which resulted in the minimum val
    AbsMinParams = mins_params[WhichZvec]
    print("\n Results from running MCMC minimization of -(SNR^2) for 3D paramter space...")
    print("\n subject to the following boundaries:")
    print("\n exposure times: [" +str(exp_bounds[0]) + ", " +str(exp_bounds[-1]) +"]" )
    print("\n individual laser powers: [" +str(pwr_bounds[0]) + ", " +str(pwr_bounds[-1]) +"]" )
    print("\n gain values: [" + str(gain_bounds[0]) + ", " + str(gain_bounds[-1]) + "]")
    print("\n with " + str(len(ZvecTotal)) + " paths and " + str(Numberiterations) + " iterations allowed per path")
    print("\n The most negative value of -(SNR^2) = " + str(AbsMin))
    print("\n The path which contained the minimum -(SNR^2): " + str(WhichZvec))
    print("\n The parameters which resulted in the minimum -(SNR^2): (exposure time, individual laser power) = " +str(AbsMinParams))
    print("\n Number of paths which saturated: " + str(NumSatPaths))
    print("\n Paths which saturated (indices of 'paths_i'): " + str(SatPaths))

Use the above defined functions to generate MCMC trajectory data and then perform data analysis on the optimization:

In [ ]:
paths_, ZvecTotal, path_iter, Numberiterations = CreateAllPaths(NumPaths=100, StepSet_exp=StepSet_exp, StepSet_pwr=StepSet_pwr, StepSet_gain=StepSet_gain, ParamSpcDims=ParamSpcDims, Numberiterations=30)

this is x0s (post T):[[7.80e-06 5.00e-06 9.60e-06 6.40e-06 8.40e-06 6.50e-06 8.10e-06 7.90e-06
  5.50e-06 9.70e-06 7.90e-06 7.40e-06 7.80e-06 6.40e-06 6.20e-06 7.10e-06
  6.90e-06 8.40e-06 5.30e-06 6.60e-06 7.60e-06 6.60e-06 4.40e-06 5.10e-06
  4.40e-06 4.70e-06 7.80e-06 9.90e-06 1.00e-05 4.00e-06 9.00e-06 8.70e-06
  9.40e-06 6.50e-06 5.40e-06 4.70e-06 9.70e-06 4.00e-06 9.20e-06 6.40e-06
  6.00e-06 8.00e-06 6.20e-06 5.50e-06 5.60e-06 8.70e-06 4.90e-06 4.00e-06
  4.50e-06 6.60e-06 4.30e-06 7.90e-06 4.00e-06 5.40e-06 6.20e-06 8.30e-06
  6.60e-06 6.80e-06 8.20e-06 8.30e-06 6.10e-06 8.40e-06 4.20e-06 8.20e-06
  6.50e-06 5.10e-06 7.10e-06 8.20e-06 6.80e-06 6.60e-06 8.40e-06 6.30e-06
  5.60e-06 5.40e-06 7.90e-06 5.30e-06 8.20e-06 6.70e-06 5.40e-06 7.50e-06
  6.60e-06 6.90e-06 9.90e-06 6.70e-06 6.50e-06 8.10e-06 1.00e-05 7.90e-06
  9.90e-06 4.80e-06 7.00e-06 8.20e-06 7.30e-06 7.60e-06 8.20e-06 7.70e-06
  6.90e-06 7.80e-06 8.10e-06 4.70e-06]
 [6.51e-05 6.41e-05 2.21e-05 6.10e-06 3.01e-05 9.31e

Initial Z value:-606.3499384875665
Plusx0: -5e-07
Plusy0: 0.0
plusq0: 0.2
Z value from step attempt 1: -572.7298148958225, with (x,y,q): (8.299999999999997e-06,6.540000000000002e-05,3.15)
Plusx1: -5e-07
Plusy1: 0.0
Plusy1: -0.2
Z value from step attempt 2: -570.7963642251204, with (x,y,q): (8.299999999999997e-06,6.540000000000002e-05,2.7499999999999996)
5e-07
-1e-07
Image is saturated
Z value from step attempt 3: 0, with (x,y,q ): (9.299999999999997e-06,6.530000000000002e-05,3.15)
4.9999999999999996e-06
1e-06
Z value from BIG JUMP final attempt: -439.14384241380037, with (x,y,q): (-6.200000000000002e-06,6.640000000000001e-05,3.3499999999999996)
this is Zvec: [[   0.        ]
 [-503.11745211]
 [-532.99821852]
 [-544.28267578]
 [-577.55773216]
 [-612.28655956]
 [-607.15013747]
 [-602.48962338]
 [-610.49782161]
 [-611.19387912]
 [-606.34993849]]
Difference in consecutive Z vals: [-4.84394063]
Initial (x,y)= (8.799999999999997e-06,6.540000000000002e-05,2.9499999999999997)
Initial Z value:-

Z value from BIG JUMP final attempt: -329.01407905252245, with (x,y,q): (4.799999999999998e-06,6.540000000000002e-05,2.549999999999999)
this is Zvec: [[   0.        ]
 [-503.11745211]
 [-532.99821852]
 [-544.28267578]
 [-577.55773216]
 [-612.28655956]
 [-607.15013747]
 [-602.48962338]
 [-610.49782161]
 [-611.19387912]
 [-606.34993849]
 [-614.61618661]
 [-641.16466538]
 [-644.76347637]
 [-650.73769715]
 [-641.7400162 ]
 [-645.84417871]
 [-675.93622517]
 [-687.70406412]
 [-681.9633665 ]]
Difference in consecutive Z vals: [-5.74069762]
Initial (x,y)= (9.799999999999998e-06,6.540000000000002e-05,2.149999999999999)
Initial Z value:-686.5915056491044
Plusx0: -5e-07
Plusy0: 0.0
plusq0: -0.2
Z value from step attempt 1: -632.9570347357417, with (x,y,q): (9.299999999999997e-06,6.540000000000002e-05,1.949999999999999)
Plusx1: 5e-07
Plusy1: 0.0
Plusy1: 0.0
Z value from step attempt 2: -643.4602960800873, with (x,y,q): (9.299999999999997e-06,6.540000000000002e-05,2.149999999999999)
0.0
-1e-07
Z va

Z value from step attempt 2: -629.8125758795932, with (x,y,q): (9.299999999999997e-06,6.430000000000002e-05,1.749999999999999)
5e-07
0.0
Z value from step attempt 3: -637.095495508805, with (x,y,q ): (9.299999999999997e-06,6.440000000000002e-05,1.549999999999999)
4.9999999999999996e-06
1e-06
Z value from BIG JUMP final attempt: -323.1793355630394, with (x,y,q): (4.799999999999998e-06,6.540000000000002e-05,1.549999999999999)
this is Zvec: [[   0.        ]
 [-503.11745211]
 [-532.99821852]
 [-544.28267578]
 [-577.55773216]
 [-612.28655956]
 [-607.15013747]
 [-602.48962338]
 [-610.49782161]
 [-611.19387912]
 [-606.34993849]
 [-614.61618661]
 [-641.16466538]
 [-644.76347637]
 [-650.73769715]
 [-641.7400162 ]
 [-645.84417871]
 [-675.93622517]
 [-687.70406412]
 [-681.9633665 ]
 [-686.59150565]
 [-676.23713956]
 [-686.04427672]
 [-678.46105679]
 [-663.81142941]
 [-678.3711927 ]
 [-670.51898308]
 [-665.48146838]]
Difference in consecutive Z vals: [-5.0375147]
Initial (x,y)= (9.799999999999998e

Initial Z value:-571.3817546131154
Plusx0: 5e-07
Plusy0: 1e-07
plusq0: 0.2
Z value from step attempt 1: -603.4889770760959, with (x,y,q): (9e-06,6.37e-05,2.95)
this is Zvec: [[-337.99387636]
 [-371.16127486]
 [-369.76686275]
 [-372.15219814]
 [-406.68195639]
 [-436.47254521]
 [-474.01417021]
 [-501.00202333]
 [-531.4048497 ]
 [-539.84186818]
 [-569.13843154]
 [-603.48897708]]
Difference in consecutive Z vals: [34.35054553]
Initial (x,y)= (9e-06,6.37e-05,2.95)
Initial Z value:-606.4378975471509
Plusx0: 5e-07
Plusy0: 1e-07
plusq0: -0.2
Image is saturated
Z value from step attempt 1: 0, with (x,y,q): (9.5e-06,6.38e-05,2.75)
Plusx1: -5e-07
Plusy1: 1e-07
Plusy1: 0.2
Z value from step attempt 2: -574.4683957980159, with (x,y,q): (8.5e-06,6.38e-05,3.1500000000000004)
5e-07
-1e-07
Z value from step attempt 3: -631.3535464347344, with (x,y,q ): (9.5e-06,6.36e-05,2.75)
this is Zvec: [[-337.99387636]
 [-371.16127486]
 [-369.76686275]
 [-372.15219814]
 [-406.68195639]
 [-436.47254521]
 [-474.01417

...

Z value from step attempt 1: -772.6870218766256, with (x,y,q): (8.900000000000001e-06,8.420000000000001e-05,0.35000000000000003)
this is Zvec: [[-377.8033467 ]
 [-386.33868042]
 [-422.55689964]
 [-431.98907736]
 [-467.61229133]
 [-516.10896434]
 [-564.12270332]
 [-604.72463949]
 [-658.05509004]
 [-693.54316333]
 [-747.04626854]
 [-772.68702188]]
Difference in consecutive Z vals: [25.64075333]
Initial (x,y)= (8.900000000000001e-06,8.420000000000001e-05,0.35000000000000003)
Initial Z value:-788.2853576007913
Plusx0: -5e-07
Plusy0: -1e-07
plusq0: -0.2
Z value from step attempt 1: -743.4947711750051, with (x,y,q): (8.400000000000001e-06,8.410000000000001e-05,0.15000000000000002)
Plusx1: -5e-07
Plusy1: 0.0
Plusy1: -0.2
Z value from step attempt 2: -736.6355702412686, with (x,y,q): (8.400000000000001e-06,8.420000000000001e-05,0.15000000000000002)
0.0
-1e-07
Z value from step attempt 3: -778.1804984751105, with (x,y,q ): (8.900000000000001e-06,8.410000000000001e-05,0.55)
-4.9999999999999996e-

Z value from step attempt 2: -301.7547029569212, with (x,y,q): (-7.899999999999999e-06,3.58e-05,4.1499999999999995)
0.0
-1e-07
Z value from step attempt 3: -314.4302290714958, with (x,y,q ): (-8.4e-06,3.57e-05,4.55)
this is Zvec: [[-194.32660461]
 [-213.25153266]
 [-229.90525787]
 [-250.49906929]
 [-272.76606679]
 [-296.14370953]
 [-318.68694969]
 [-317.10681746]
 [-314.43022907]]
Difference in consecutive Z vals: [-2.67658839]
Initial (x,y)= (-8.4e-06,3.57e-05,4.55)
Initial Z value:-316.88144860875235
Plusx0: -5e-07
Plusy0: 0.0
plusq0: 0.2
Z value from step attempt 1: -298.9104399128768, with (x,y,q): (-7.899999999999999e-06,3.57e-05,4.75)
Plusx1: 0.0
Plusy1: 1e-07
Plusy1: 0.2
Z value from step attempt 2: -317.888090567994, with (x,y,q): (-8.4e-06,3.58e-05,4.75)
this is Zvec: [[-194.32660461]
 [-213.25153266]
 [-229.90525787]
 [-250.49906929]
 [-272.76606679]
 [-296.14370953]
 [-318.68694969]
 [-317.10681746]
 [-314.43022907]
 [-317.88809057]]
Difference in consecutive Z vals: [3.4578

Z value from step attempt 1: -186.43296922734686, with (x,y,q): (5.399999999999999e-06,3.3500000000000015e-05,1.2000000000000002)
this is Zvec: [[-146.56839799]
 [-148.66710466]
 [-150.85206776]
 [-166.48120526]
 [-167.00334217]
 [-182.37044106]
 [-184.45161626]
 [-185.09302048]
 [-186.43296923]]
Difference in consecutive Z vals: [1.33994874]
Initial (x,y)= (5.399999999999999e-06,3.3500000000000015e-05,1.2000000000000002)
Initial Z value:-185.39711436722598
Plusx0: 0.0
Plusy0: -1e-07
plusq0: -0.2
Z value from step attempt 1: -187.4018031881216, with (x,y,q): (5.399999999999999e-06,3.340000000000001e-05,1.0000000000000002)
this is Zvec: [[-146.56839799]
 [-148.66710466]
 [-150.85206776]
 [-166.48120526]
 [-167.00334217]
 [-182.37044106]
 [-184.45161626]
 [-185.09302048]
 [-186.43296923]
 [-187.40180319]]
Difference in consecutive Z vals: [0.96883396]
Initial (x,y)= (5.399999999999999e-06,3.340000000000001e-05,1.0000000000000002)
Initial Z value:-181.57661332400326
Plusx0: 0.0
Plusy0: 1e

Z value from step attempt 2: -382.2557314788194, with (x,y,q): (-1.11e-05,3.320000000000001e-05,0.8000000000000003)
0.0
-1e-07
Z value from step attempt 3: -395.6175870313068, with (x,y,q ): (-1.16e-05,3.320000000000001e-05,0.8000000000000003)
4.9999999999999996e-06
0.0
Z value from BIG JUMP final attempt: -579.6857267406999, with (x,y,q): (-1.66e-05,3.330000000000001e-05,0.6000000000000003)
this is Zvec: [[-146.56839799]
 [-148.66710466]
 [-150.85206776]
 [-166.48120526]
 [-167.00334217]
 [-182.37044106]
 [-184.45161626]
 [-185.09302048]
 [-186.43296923]
 [-187.40180319]
 [-183.78141555]
 [-332.84015869]
 [-335.01226429]
 [-357.61091771]
 [-353.78133131]
 [-351.18278436]
 [-370.06277797]
 [-381.74760145]
 [-401.91477648]
 [-401.29572521]
 [-410.25339746]
 [-579.68572674]]
Difference in consecutive Z vals: [169.43232928]
Initial (x,y)= (-1.66e-05,3.330000000000001e-05,0.6000000000000003)
Initial Z value:-581.9428877497703
Plusx0: 5e-07
Plusy0: 1e-07
plusq0: -0.2
Z value from step attem

plusq0: -0.2
Z value from step attempt 1: -390.43020651989343, with (x,y,q): (4.699999999999999e-06,7.920000000000001e-05,2.45)
this is Zvec: [[-390.48544423]
 [-390.43020652]]
Break condition met, negligible difference between consecutive Z values

Current Coords: [[4.70e-06 7.92e-05 2.45e+00]], Zexit: [[-390.43020652]], iterations performed: 0

Difference in consecutive Z vals: [-0.05523771]
Image is saturated
Initial (x,y)= (7.799999999999996e-06,9.810000000000001e-05,4.7)
Image is saturated
Initial Z value:0
Plusx0: 0.0
Plusy0: 1e-07
plusq0: 0.0
Image is saturated
Z value from step attempt 1: 0, with (x,y,q): (7.799999999999996e-06,9.820000000000002e-05,4.7)
this is Zvec: [[0]
 [0]]
Break condition met, negligible difference between consecutive Z values

Current Coords: [[7.80e-06 9.82e-05 4.70e+00]], Zexit: [[0]], iterations performed: 0

Difference in consecutive Z vals: [0]
Image is saturated
Initial (x,y)= (9.899999999999993e-06,7.610000000000002e-05,2.6500000000000004)
Image i

In [45]:
SNR_Analysis(ZvecTotal=ZvecTotal, paths_=paths_, exp_bounds=exp_bounds, pwr_bounds=pwr_bounds, gain_bounds=gain_bounds)


 Results from running MCMC minimization of -(SNR^2) for 3D paramter space...

 subject to the following boundaries:

 exposure times: [4e-06, 9.999999999999992e-06]

 individual laser powers: [1e-07, 9.910000000000002e-05]

 gain values: [0.0, 4.95]

 with 100 paths and 30 iterations allowed per path

 The most negative value of -(SNR^2) = -1209.2040969542759

 The path which contained the minimum -(SNR^2): 23

 The parameters which resulted in the minimum -(SNR^2): (exposure time, individual laser power) = [ 1.81e-05  6.29e-05 -7.50e-01]

 Number of paths which saturated: 11

 Paths which saturated (indices of 'paths_i'): [2, 4, 12, 18, 26, 30, 49, 57, 58, 68, 78]


The code below computes the standard deviation of the FOM when given the same input parameters on each calculation:

In [42]:
#This code is to compute the standard deviation of calculating the same -<SNR>^2 for 'testArraylen'
#number of times
testArray=[]
testArraylen=100
for i in range(testArraylen):
    testVal = ParamChoices_SNR(exposure_time=5e-6,power=1e-4, gain_val= 1)
    #suppress this print statement if you don't need to see all the values
    print("SNR_i: " +str(testVal) + ", i: " +str(i))
    testArray.append(testVal)

testMean = sum(testArray)/len(testArray)

Numerator = []
for i in range(len(testArray)):
    diffSq = (testArray[i]-testMean)**2
    Numerator.append(diffSq)

testDiffSquaredSum = sum(Numerator)
testSampleStdDev = np.sqrt(testDiffSquaredSum/(len(testArray)-1))
print("average: "+str(testMean) + ", stddev: "+str(testSampleStdDev))

SNR_i: -521.7983486732408, i: 0
SNR_i: -532.5104612559144, i: 1
SNR_i: -518.3365553251202, i: 2
SNR_i: -528.359540555488, i: 3
SNR_i: -524.3771663885225, i: 4
SNR_i: -528.2629658459538, i: 5
SNR_i: -528.0591794822915, i: 6
SNR_i: -523.8480109192184, i: 7
SNR_i: -520.9570118995034, i: 8
SNR_i: -529.1234752238886, i: 9
SNR_i: -534.7274788544026, i: 10
SNR_i: -523.9397146636867, i: 11
SNR_i: -521.3728223061249, i: 12
SNR_i: -527.7964650239123, i: 13
SNR_i: -517.0113466219499, i: 14
SNR_i: -520.0401061773313, i: 15
SNR_i: -516.618553025523, i: 16
SNR_i: -526.0564893698199, i: 17
SNR_i: -525.0893680615654, i: 18
SNR_i: -525.3006557007111, i: 19
SNR_i: -518.4540694019423, i: 20
SNR_i: -531.893674089337, i: 21
SNR_i: -527.1056156509021, i: 22
SNR_i: -524.8700163547884, i: 23
SNR_i: -521.5565872229104, i: 24
SNR_i: -520.4934510275571, i: 25
SNR_i: -523.4978119597305, i: 26
SNR_i: -532.0849857846517, i: 27
SNR_i: -520.7908127592121, i: 28
SNR_i: -519.8593954740456, i: 29
SNR_i: -533.08453974872